In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from crewai.tools import BaseTool
from pydantic import BaseModel

In [ ]:
local_llm = LLM(
    model="ollama/llama3.1",
    base_url="http://localhost:11434"
)

cloud_llm = LLM(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)


In [ ]:
# Initialize the fools

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
# Agents

# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue based on event requirements",
    backstory="""
    With a keen sense of space and understanding of event logistics,
    you excel at finding and securing the perfect venue that fits the
    event's theme, size, and budget constraints.
    """,
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=cloud_llm
)

# Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal="Manage all logistics for the event including catering and equipment setup",
    backstory="""
    Organized and detail-oriented, you ensure that every logistical aspect of the event 
    from catering to equipment setup is flawlessly executed to create a seamless experience.
    """,
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=cloud_llm
)

# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and communicate with participants",
    backstory="""
    Creative and communicative, you craft compelling messages and 
    engage with potential attendees to maximize event exposure and participation.
    """,
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=local_llm
)


In [ ]:
# Create a pydantic object to store the venue details
# Agents can create instances and populate this model

class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str



#### Tasks

- By using output_json, you can specify the structure of the output you want.
- By using output_file, you can get your output in a file.
- By setting human_input=True, the task will ask for human feedback (whether you like the results or not) before finalising it.
- By setting async_execution=True, it means the task can run in parallel with the tasks which come after it.

In [ ]:
# Tasks

venue_task = Task(
    description="""
    Find a {venue_type} in {event_city} that meets criteria for {event_topic} and {event_description}
    and can hold more than {expected_participants} participants.
    Find availability of the venue on {tentative_date}.
    """,
    expected_output="""
    All the details of a specifically chosen venue you found to accommodate the event.
    """,
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

logistics_task = Task(
    description="""
    Coordinate catering that can serve {food_preference} and equipment for an event {event_topic} 
    with {expected_participants} participants on {tentative_date} at {event_city}.
    """,
    expected_output="""
    Confirmation of all logistics arrangements including catering and equipment setup.
    """,
    human_input=True,
    async_execution=True, # This taks can execute in parallel
    agent=logistics_manager
)

marketing_task = Task(
    description="""
    Promote the {event_topic} for {event_description} aiming to engage at 
    least{expected_participants} potential attendees at {event_city} on {tentative_date}.
    """,
    expected_output="""
    Report on marketing activities and attendee engagement formatted as markdown.
    """,
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

he crew must end with at most one asynchronous task occurs because CrewAI requires a definitive "end" to the process to know when to return the final result.

In our code, you have defined both logistics_task and marketing_task as asynchronous (async_execution=True) and placed them at the end of the list:

When the Crew executes:

It runs venue_task.
It starts logistics_task (async) and moves on immediately.
It starts marketing_task (async) and moves on immediately.
If the last task is also asynchronous, the Crew would technically "finish" its execution steps instantly without waiting for the actual work to complete, leaving no clear final output to return. CrewAI enforces that at least the final task (or the last one in the chain) must be synchronous so the system knows to wait for it to complete.

Add a final summary task If you absolutely need both logistics and marketing to run in parallel, add a fourth task at the end that is synchronous. This task could simply aggregate the results from the previous tasks.

In [ ]:
summary_task = Task(
    description="Compile the results from logistics and marketing.",
    expected_output="A final event plan summary.",
    agent=venue_coordinator, # or another agent
    async_execution=False
)

In [ ]:
# Crew

event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task, summary_task], # Since logistics_task and marketing_task are async, the order of tasks doesn't matter
    verbose=True
)

# Run the crew


In [ ]:
# inputs

event_details = {
    'event_topic': "House Warming Ceremony",
    'event_description': """
    A gathering of mostly Indian friends and family to celebrate the new homeowner's move into their new house.
    """,
    'event_city': "Ebbsfleet Valley, Kent, UK",
    'tentative_date': "18-January-2026",
    'expected_participants': 100,
    'food_preference': "Indian",
    'budget': 1500,
    'venue_type': "Community Center"
}

In [ ]:
result = event_management_crew.kickoff(inputs= event_details)